In [1]:
import re
import math

from copy import deepcopy
from itertools import cycle, combinations, permutations
from collections import Counter, defaultdict, deque
from io import StringIO

def read_input(day, fn=str.strip):
    """
    Return a list of the input lines mapped by fn
    
    example: 
    >>> read_input('01', int)  # read input file, map all lines to int
    
    Inspired by Peter Norvig: https://github.com/norvig/pytudes
    
    """
    return list(map(fn, open(f'input\{day}.txt')))

def all_integers(s):
    """return all integers from a string"""
    return tuple(map(int, re.findall(r'-?\d+', s)))

# day 12

In [2]:
testcase = """<x=-1, y=0, z=2>
<x=2, y=-10, z=-7>
<x=4, y=-8, z=8>
<x=3, y=5, z=-1>""".split('\n')

In [3]:
all_integers(testcase[0])

(-1, 0, 2)

In [4]:
class Moon(object):
    def __init__(self, line):
        self.x, self.y, self.z = all_integers(line)
        self.vx, self.vy, self.vz = 0, 0, 0
        
    def __repr__(self):
        return f'pos=<x={self.x}, y={self.y}, z={self.z}>, vel=<x={self.vx}, y={self.vy}, z={self.vz}>'
    
    def calc_velocity(self, other_moons):
        for other_moon in other_moons:
            self.vx += other_moon.x > self.x
            self.vy += other_moon.y > self.y
            self.vz += other_moon.z > self.z
            self.vx -= other_moon.x < self.x
            self.vy -= other_moon.y < self.y
            self.vz -= other_moon.z < self.z

    def calc_position(self):
        self.x += self.vx
        self.y += self.vy
        self.z += self.vz
        
    def calc_energy(self):
        pe = abs(self.x) + abs(self.y) + abs(self.z)
        ke = abs(self.vx) + abs(self.vy) + abs(self.vz)
        return ke * pe
        

m = Moon(testcase[0])
n = Moon(testcase[1])
print(m, n)
m.calc_velocity([n])
n.calc_velocity([m])
print(m, n)
n.calc_energy()


pos=<x=-1, y=0, z=2>, vel=<x=0, y=0, z=0> pos=<x=2, y=-10, z=-7>, vel=<x=0, y=0, z=0>
pos=<x=-1, y=0, z=2>, vel=<x=1, y=-1, z=-1> pos=<x=2, y=-10, z=-7>, vel=<x=-1, y=1, z=1>


57

In [5]:
def partA(l, n=10):
    moons = [Moon(line) for line in l]

    for t in range(n):
        #print(f'after {t}')
        for moon in moons:
            moon.calc_velocity(moons)
        for moon in moons:
            moon.calc_position()
            #print(moon)

    for moon in moons:
        print(moon, moon.calc_energy())

    return sum(m.calc_energy() for m in moons)

partA(testcase)

pos=<x=2, y=1, z=-3>, vel=<x=-3, y=-2, z=1> 36
pos=<x=1, y=-8, z=0>, vel=<x=-1, y=1, z=3> 45
pos=<x=3, y=-6, z=1>, vel=<x=3, y=2, z=-3> 80
pos=<x=2, y=0, z=4>, vel=<x=1, y=-1, z=-1> 18


179

In [79]:
testcase2 = """<x=-8, y=-10, z=0>
<x=5, y=5, z=10>
<x=2, y=-7, z=3>
<x=9, y=-8, z=-3>""".split('\n')

partA(testcase2, n=100)

[((8, -12, -9), (-7, 3, 0)), ((13, 16, -3), (3, -11, -5)), ((-29, -11, -1), (-3, 7, 4)), ((16, -13, 23), (7, 1, 1))]


1940

In [80]:
inp = """<x=17, y=-9, z=4>
<x=2, y=2, z=-13>
<x=-1, y=5, z=-1>
<x=4, y=7, z=-7>""".split('\n')

partA(inp, n=1000)

[((21, 44, 4), (-15, -12, 9)), ((-49, 1, -17), (-2, 6, 4)), ((-13, -10, 22), (0, 1, -4)), ((63, -30, -26), (17, 5, -9))]


7202

# part A without classes

In [8]:
testcase = """<x=-1, y=0, z=2>
<x=2, y=-10, z=-7>
<x=4, y=-8, z=8>
<x=3, y=5, z=-1>""".split('\n')

In [9]:
moons = [(all_integers(line), (0,0,0)) for line in testcase]

In [10]:
moons

[((-1, 0, 2), (0, 0, 0)),
 ((2, -10, -7), (0, 0, 0)),
 ((4, -8, 8), (0, 0, 0)),
 ((3, 5, -1), (0, 0, 0))]

In [29]:
def calc_velocity(x, v, other_moons, dimensions=range(3)):
    v = list(v)
    for x_o, v_o in other_moons:
        for idx in dimensions:
            v[idx] += x_o[idx] > x[idx]
            v[idx] -= x_o[idx] < x[idx]
    return (x, tuple(v))

def calc_position(x, v, dimensions=range(3)):
    x = list(x)
    for idx in dimensions:
        x[idx] += v[idx]
    return (tuple(x), v)

def calc_energy(x, v):
    pe = sum(map(abs, x))
    ke = sum(map(abs, v))
    return ke * pe

def partA(l, n=10):
    moons = [(all_integers(line), (0,0,0)) for line in l]

    for t in range(n):
        #print(f'after {t}')
        moons = [calc_velocity(*moon, moons) for moon in moons]
        moons = [calc_position(*moon) for moon in moons]     
    
    print(moons)

    return sum(calc_energy(*moon) for moon in moons)

partA(testcase)
            

[((2, 1, -3), (-3, -2, 1)), ((1, -8, 0), (-1, 1, 3)), ((3, -6, 1), (3, 2, -3)), ((2, 0, 4), (1, -1, -1))]


179

In [14]:
inp = """<x=17, y=-9, z=4>
<x=2, y=2, z=-13>
<x=-1, y=5, z=-1>
<x=4, y=7, z=-7>""".split('\n')

partA(inp, n=1000)

[((21, 44, 4), (-15, -12, 9)), ((-49, 1, -17), (-2, 6, 4)), ((-13, -10, 22), (0, 1, -4)), ((63, -30, -26), (17, 5, -9))]


7202

# find loop (part B)

In [71]:
def find_loop(l, n=10, D=range(3)):
    moons = [(all_integers(line), (0,0,0)) for line in l]
    seen = {tuple(moons)}
    
    for t in range(1, n+1):
        #print(f'after {t}')
        moons = [calc_velocity(*moon, moons, dimensions=D) for moon in moons]
        moons = [calc_position(*moon,dimensions=D) for moon in moons]     
        if tuple(moons) in seen:
            print('found: ', D, t)
            return t
        seen.add(tuple(moons))
    
find_loop(testcase, n=3000)

found:  range(0, 3) 2772


2772

In [72]:
find_loop(testcase, n=1000, D=[0])
D = [18, 28, 44]

found:  [0] 18


In [73]:
from math import gcd
from functools import reduce

In [74]:
def LCM(a, b):
    return int(abs(a*b)/gcd(a,b))

LCM(21, 6) # wikipedia example

42

In [75]:
t = [2, 3, 4, 5, 7]  # wikipedia example: 420
reduce(LCM, t)

420

In [76]:
reduce(LCM, D)

2772

In [86]:
def partB(l):
    """find the length of the loop per dimension (x, y, z) and reduce using LCM""" 
    n = [find_loop(l, n=int(1e9), D=[dimension]) for dimension in range(3)]
    return reduce(LCM, n)

partB(testcase)
        

found:  [0] 18
found:  [1] 28
found:  [2] 44


2772

In [87]:
partB(testcase2)

found:  [0] 2028
found:  [1] 5898
found:  [2] 4702


4686774924

In [88]:
partB(inp)

found:  [0] 231614
found:  [1] 96236
found:  [2] 193052


537881600740876